In [1]:
# ruff: noqa: F401, E402, T201, D103

from __future__ import annotations

import math
import sys
import time
from bisect import bisect_left, bisect_right
from collections import Counter, deque
from functools import cmp_to_key, lru_cache, reduce
from heapq import heapify, heappop, heappush, heappushpop, nsmallest
from importlib import reload
from itertools import (
    accumulate,
    chain,
    combinations,
    islice,
    pairwise,
    permutations,
    product,
    repeat,
    starmap,
    tee,
)
from math import comb, exp, factorial, inf, log, prod, sqrt
from operator import add, and_, contains, indexOf, itemgetter, mul, neg, or_, xor
from pprint import pprint
from typing import Callable, Iterable, List, Tuple

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from multiprocess import Process

import arrays
import combinatorics
import graphs
import lists
import mathematics
import matrix
import parsing
import search
import sequences
import sets
import stack
import strings
import trees

for m in (
    arrays,
    combinatorics,
    graphs,
    lists,
    mathematics,
    matrix,
    parsing,
    search,
    sequences,
    sets,
    stack,
    strings,
    trees,
):
    reload(m)


from combinatorics import fib
from search import lower_index, upper_index
from sequences import find_if
from strings import splint

print()


In [17]:
from trees import Node, TreeNode


def number_of_turns(r: Node, a: int, b: int) -> int:
    def lca(n: Node) -> Node:
        if n is None or n.data in (a, b):
            return n
        l, r = lca(n.left), lca(n.right)
        return l and r and n or l or r

    def dfs(n: Node, target: int, d: str) -> int:
        if n is None:
            return -1
        if n.data == target:
            return 0
        lt = dfs(n.left, target, "L")
        if lt >= 0:
            return lt + (d == "R")
        rt = dfs(n.right, target, "R")
        if rt >= 0:
            return rt + (d == "L")
        return -1

    if r is None or a == b:
        return -1
    ca = lca(r)
    if ca is None:
        return -1

    return dfs(ca, a, "") + dfs(ca, b, "") + (ca.data not in (a, b))


In [18]:
t = trees.make("1 2 3 4 5 6 7")
t.display()
number_of_turns(t, 5, 6)


  _1_  
 /   \ 
 2   3 
/ \ / \
4 5 6 7


3

In [19]:
t = trees.make("1 2 3")
t.display()
number_of_turns(t, 2, 3)


 1 
/ \
2 3


1

In [20]:
t = trees.make("1 2 3")
t.display()
number_of_turns(t, 1, 2)


 1 
/ \
2 3


0

In [14]:
t = trees.make("1 2 N N 4")
t.display()
number_of_turns(t, 1, 4)


 _1
/  
2  
 \ 
 4 


1

In [15]:
t = trees.make("1 2 3 N 4 N N 5")
t.display()
number_of_turns(t, 1, 5)


 __1 
/   \
2_  3
  \  
  4  
 /   
 5   


2

In [16]:
t = trees.make("1 2 3 4 5 6 7 8 N N N 9 10")
t.display()
number_of_turns(t, 5, 10)


   _1____  
  /      \ 
  2    __3 
 / \  /   \
 4 5  6_  7
/    /  \  
8    9 10  


4